In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, datasets
import torch.multiprocessing as _mp
import pandas as pd
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset

import resnet_cifar10
from tqdm import tqdm

from networks import *
from utils import *

import copy
import time
import pickle

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [2]:
er = [[-1], [], []]
print(er)
print(not er[0])

[[-1], [], []]
False


### RESNET

In [3]:
'''
test_transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))])

test_set = datasets.CIFAR10("cifar10", train=False, download=True, transform=test_transform)
testloaders = torch.utils.data.DataLoader(test_set, batch_size=256, shuffle=False)
'''

'\ntest_transform = transforms.Compose(\n    [transforms.ToTensor(),\n     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))])\n\ntest_set = datasets.CIFAR10("cifar10", train=False, download=True, transform=test_transform)\ntestloaders = torch.utils.data.DataLoader(test_set, batch_size=256, shuffle=False)\n'

In [4]:
'''
model_type = "20"
models = []
ens_num = [0, 301]
model_name ="resnet"+str(model_type)
models_path = "resnet_models/"

# load all models
models = eval_model_load(model_type, ens_num, model_name, models_path)

print("Model Loaded")
print("Acc", ens_evaluate_acc(models=models, testloaders=testloaders))

models_time = []
for i in range(5):
    models_time.append(ens_evaluate_acc(models=models, testloaders=testloaders, single=True, time_profiler=True)[-1])
print(models_time)
'''

'\nmodel_type = "20"\nmodels = []\nens_num = [0, 301]\nmodel_name ="resnet"+str(model_type)\nmodels_path = "resnet_models/"\n\n# load all models\nmodels = eval_model_load(model_type, ens_num, model_name, models_path)\n\nprint("Model Loaded")\nprint("Acc", ens_evaluate_acc(models=models, testloaders=testloaders))\n\nmodels_time = []\nfor i in range(5):\n    models_time.append(ens_evaluate_acc(models=models, testloaders=testloaders, single=True, time_profiler=True)[-1])\nprint(models_time)\n'

### EfficientNet

In [5]:
# test_transform = transforms.Compose(
#     [transforms.Resize(224),
#      transforms.ToTensor(),
#      transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))])

# test_set = datasets.CIFAR10("cifar10", train=False, download=True, transform=test_transform)
# testloaders = torch.utils.data.DataLoader(test_set, batch_size=32, shuffle=False)

In [6]:
# from torchvision.models import efficientnet_b0, efficientnet_b1

# model_type = 0
# models = []
# ens_num = [5, 7]

# model_name ="eb"+str(model_type)
# models_path = "efficientnet_models/"

# # load all models
# m = len(ens_num)
# for i in range(m):   
#     model_name_path = model_name +"-"+str(ens_num[i])
#     if model_type==0:
#         model = efficientnet_b0()
#         model.classifier[1] = nn.Linear(1280, 10)
#     if model_type==1:
#         model = efficientnet_b1()
#         model.classifier[1] = nn.Linear(1280, 10)
#     print("LOAD",models_path + model_name_path + ".pt")
#     model.load_state_dict(torch.load(models_path + model_name_path + ".pt", map_location=device))
#     model.eval()
#     model.to(device)
#     models.append(model)

# print("Model Loaded")
# print("Acc", ens_evaluate_acc(models, testloaders))

### Small (for MILR Benchmark)

In [7]:
# test_transform = transforms.Compose(
#     [transforms.ToTensor(),
#      transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))])

# test_set = datasets.CIFAR10("cifar10", train=False, download=True, transform=test_transform)
# testloaders = torch.utils.data.DataLoader(test_set, batch_size=256, shuffle=False)

In [8]:
# model_type = "smallmilr"
# models = []
# ens_num = [0, 1]
# model_name ="vsmilr4-100"
# models_path = "MILR/"

# # load all models
# models = eval_model_load(model_type, ens_num, model_name, models_path)


### DistilBert

In [9]:
''''''
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

url = "https://raw.githubusercontent.com/Mohamed2519/Text-Classification-For-SST2-dataset/main/data/test.tsv"
df = pd.read_csv(url, delimiter='\t', names=['labels','sentence'])

encoded_data = tokenizer.batch_encode_plus(
    df['sentence'].tolist(),
    add_special_tokens=True,
    max_length=128,
    padding='max_length',
    truncation=True,
    return_attention_mask=True,
    return_tensors='pt'
)

# Create TensorDataset from encoded_data
dataset = TensorDataset(
    encoded_data['input_ids'].to(device),
    encoded_data['attention_mask'].to(device),
    torch.tensor(df['labels'].tolist()).to(device)
)

# Create DataLoader from TensorDataset
batch_size = 32
testloaders = DataLoader(dataset, batch_size=batch_size, shuffle=False)


In [10]:
''''''
model_type = "DistilBert"
models = []
ens_num = [7, 12]
model_name ="transformer"+str(model_type)
models_path = "distilbert_models/"

# load all models
models = eval_model_load(model_type, ens_num, model_name, models_path)

print("Model Loaded")
print("Acc", ens_evaluate_acc(model_type=model_type, models=models, testloaders=testloaders))

models_time = []
for i in range(5):
    models_time.append(ens_evaluate_acc(model_type=model_type, models=models, testloaders=testloaders, single=True, time_profiler=True)[-1])
print(models_time)


LOAD distilbert_models/transformerDistilBert-7.pt
LOAD distilbert_models/transformerDistilBert-12.pt
Model Loaded
Acc 0.9247666117517848
TIME model 0: 15.01 ms
TIME model 1: 8.24 ms
TIME ALL: 23.25 ms
TIME model 0: 6.57 ms
TIME model 1: 8.12 ms
TIME ALL: 14.69 ms
TIME model 0: 9.69 ms
TIME model 1: 8.58 ms
TIME ALL: 18.27 ms
TIME model 0: 7.91 ms
TIME model 1: 6.05 ms
TIME ALL: 13.96 ms
TIME model 0: 6.02 ms
TIME model 1: 7.59 ms
TIME ALL: 13.61 ms
[[15.012264251708984, 8.237361907958984, 23.24962615966797], [6.56890869140625, 8.124828338623047, 14.693737030029297], [9.693384170532227, 8.578777313232422, 18.27216148376465], [7.911443710327148, 6.047725677490234, 13.959169387817383], [6.01959228515625, 7.593870162963867, 13.613462448120117]]


# GET THE SUM and DELTA

In [11]:
deltas, summs = get_sum_delta(models)

# Evaluate

In [12]:
modelDelta = NetDelta(models, ens_num, model_type, deltas, summs)
modelDelta.to(device)
modelDelta.eval()
print("model ready")
print("Acc", ens_evaluate_acc(model_type=model_type, wrappermodel=modelDelta, testloaders=testloaders))

model ready
Acc 0.9247666117517848


In [13]:
x = torch.tensor(-0.04562711715698242)
y = torch.tensor(-0.04562878608703613)
print(torch.isclose(x,y, rtol=1e-4))
print((1e-03 + 1e-08)*abs(x))

tensor(True)
tensor(4.5628e-05)


# Pytorch Profiler

In [14]:
%colors nocolor

times = []
for k in range(1):
    modelDelta.debug = False
    modelDelta.use_sum = True
    modelDelta.eval()
    modelDelta = modelDelta.to(device)

    # warmup
    ens_evaluate_acc(model_type=model_type, wrappermodel=modelDelta, testloaders=testloaders)
    ens_evaluate_acc(model_type=model_type, wrappermodel=modelDelta, testloaders=testloaders, single=True, time_profiler=True)

    print("ORIGINAL")
    print(ens_evaluate_acc(model_type=model_type, wrappermodel=modelDelta, testloaders=testloaders)) 
    ens_evaluate_acc(model_type=model_type, wrappermodel=modelDelta, testloaders=testloaders, single=True, time_profiler=True)
    print("==================")

    
    modelDelta.debug = False
    for t in range(1): # evaluate fixing time
        print(f"\nBREAK {t}")
        modelDelta.use_sum = False
        modelDelta.models[0] = flip(modelDelta.models[0], n=642, debug=False)
        print(ens_evaluate_acc(model_type=model_type, wrappermodel=modelDelta, testloaders=testloaders))
        print("==================")

        # FIXING
        print(f"\nFIXING {t}")
        modelDelta.use_sum = True
        modelDelta.skipper = True
        modelDelta.debug = False
        modelDelta.direct_recovery = False

        _, _, times = ens_evaluate_acc(model_type=model_type, wrappermodel=modelDelta, testloaders=testloaders, single=True, time_profiler=True)
        
        modelDelta.recover_recent(time_profiler=True)
        # print(ens_evaluate_acc(wrappermodel=modelDelta, testloaders=testloaders))

        # while modelDelta.errors[0]!=-1:
        #     print(modelDelta.errors)
        
        #     modelDelta.recover_recent()

        #     modelDelta.debug = False
        #     _, _, times = ens_evaluate_acc(wrappermodel=modelDelta, testloaders=testloaders, single=True, time_profiler=True)
        #     # print(ens_evaluate_acc(wrappermodel=modelDelta, testloaders=testloaders))
        #     print("==================")

TIME model 0: 77.638 ms
TIME model 1: 72.115 ms
TIME ALL: 150.75 ms
ORIGINAL
0.9247666117517848
TIME model 0: 77.627 ms
TIME model 1: 74.145 ms
TIME ALL: 152.77 ms

BREAK 0
0.9225700164744646

FIXING 0
TIME model 0: 46.072 ms
TIME model 1: 74.193 ms
TIME ALL: 120.27 ms
recovering...bismillah
TIME recovery: 2.497 ms


In [15]:
modelDelta.debug = False
print(ens_evaluate_acc(model_type=model_type, wrappermodel=modelDelta, testloaders=testloaders))
print("==================")

0.9231191652937946
